In [1]:
import os
import xml
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from nni.compression.pytorch import ModelSpeedup
from nni.compression.pytorch.utils.counter import count_flops_params

from utils import create_model, get_dataloader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_workers = 16
torch.set_num_threads(num_workers)

### Stanford Dogs (Data Download)

The [Stanford Dogs](http://vision.stanford.edu/aditya86/ImageNetDogs/) dataset contains images of 120 breeds of dogs from around the world. It is built using images and annotation from ImageNet for the task of fine-grained image classification. We choose this task to simulate a transfer learning scenario, where a model pre-trained on the ImageNet is further transferred to an often simpler downstream task.

To download and prepare the data, please run `prepare_data.sh`, which downloads the images and annotations, and preprocesses the images for training.

In [ ]:
# Run prepare_data.sh
!chmod u+x prepare_data.sh
!./prepare_data.sh

 Then, you may run following code block, which shows several instances:

# Model Pre-training
First, we obtain a MobileNetV2 model on this task, which will serve as the base model for compression. Unfortunately, although this step is often called model "pre-training" in the model compression teminologies, we are actually finetuning a model pre-trained on ImageNet. 

In [ ]:
%run pretrain.py --model_type mobilenet_v2_torchhub 

In [5]:
%run pretrain.py --model_type resnet18

Namespace(model_type='resnet18', experiment_dir=None, checkpoint_name='checkpoint_best.pt', n_workers=16, n_epochs=180, learning_rate=0.0001, weight_decay=0.0, batch_size=64)


Using cache found in /home/flo/.cache/torch/hub/pytorch_vision_v0.8.1


Loading checkpoint from pretrained_resnet18/checkpoint_best.pt
Start training epoch 0


100%|██████████| 19/19 [00:01<00:00, 12.03it/s]


Epoch 0: train loss 0.1743, valid loss 0.5889, valid acc 0.8265
Start training epoch 1


...

Epoch 177: train loss 0.0736, valid loss 0.6120, valid acc 0.8174
Start training epoch 178


100%|██████████| 19/19 [00:01<00:00, 12.32it/s]


Epoch 178: train loss 0.0720, valid loss 0.6149, valid acc 0.8183
Start training epoch 179


100%|██████████| 19/19 [00:01<00:00, 13.59it/s]


Epoch 179: train loss 0.0720, valid loss 0.6110, valid acc 0.8174


In [6]:
%run pretrain.py --model_type efficientnet_se_b0

Namespace(model_type='efficientnet_se_b0', experiment_dir=None, checkpoint_name='checkpoint_best.pt', n_workers=16, n_epochs=180, learning_rate=0.0001, weight_decay=0.0, batch_size=64)


Using cache found in /home/flo/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Loading checkpoint from pretrained_efficientnet_se_b0/checkpoint_best.pt
Start training epoch 0


100%|██████████| 19/19 [00:01<00:00, 10.59it/s]


Epoch 0: train loss 0.2929, valid loss 0.4733, valid acc 0.8457
Start training epoch 1


...

100%|██████████| 19/19 [00:01<00:00, 10.63it/s]


Epoch 80: train loss 0.1969, valid loss 0.4568, valid acc 0.8517
Start training epoch 81


100%|██████████| 19/19 [00:01<00:00, 10.59it/s]


Epoch 81: train loss 0.1965, valid loss 0.4564, valid acc 0.8509
Start training epoch 82


 63%|██████▎   | 107/169 [00:23<00:13,  4.67it/s]

In [4]:
%run pretrain.py --model_type efficientnet_b0 --n_epochs 100

Namespace(model_type='efficientnet_b0', experiment_dir=None, checkpoint_name='checkpoint_best.pt', n_workers=16, n_epochs=100, learning_rate=0.0001, weight_decay=0.0, batch_size=64)


Using cache found in /home/flo/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Loading checkpoint from pretrained_efficientnet_b0/checkpoint_best.pt
Start training epoch 0


100%|██████████| 19/19 [00:01<00:00, 10.01it/s]


Epoch 0: train loss 0.8443, valid loss 0.7732, valid acc 0.8037
Start training epoch 1


...

Epoch 97: train loss 0.4840, valid loss 0.5296, valid acc 0.8363
Start training epoch 98


100%|██████████| 19/19 [00:02<00:00,  6.52it/s]


Epoch 98: train loss 0.4825, valid loss 0.5283, valid acc 0.8374
Start training epoch 99


100%|██████████| 19/19 [00:02<00:00,  8.26it/s]


Epoch 99: train loss 0.4733, valid loss 0.5279, valid acc 0.8380
